## 1.缓解组与未缓解组基线水平临床量表差异性检验

In [1]:
import pandas as pd
from scipy import stats

file_path = 'rawdata/缓解与未缓解.xlsx'
data = pd.read_excel(file_path)

columns_to_compare = [
    'PANSS-N(1)', 'PANSS-P(1)', 'PANSS-G(1)', 'PANSS-T(1)',
    'Baseline_Negative', 'Baseline_Positive', 'Baseline_Affective', 'Baseline_Cognitive'
]

group1 = data[data['Group'] == 1]
group2 = data[data['Group'] == 2]

normality_results = {}
homogeneity_results = {}

for column in columns_to_compare:
    normality_results[column] = {
        'Group 1': stats.shapiro(group1[column]),
        'Group 2': stats.shapiro(group2[column])
    }
    

    stat, p_value = stats.levene(group1[column], group2[column])
    homogeneity_results[column] = {'Levene Statistic': stat, 'p-value': p_value}

print("Normality Test Results (Shapiro-Wilk):")
for column in normality_results:
    print(f"{column}:")
    print(f"    Group 1: W-statistic={normality_results[column]['Group 1'][0]:.3f}, p-value={normality_results[column]['Group 1'][1]:.3f}")
    print(f"    Group 2: W-statistic={normality_results[column]['Group 2'][0]:.3f}, p-value={normality_results[column]['Group 2'][1]:.3f}")

print("\nHomogeneity of Variance Test Results (Levene's Test):")
for column in homogeneity_results:
    print(f"{column}: Levene Statistic = {homogeneity_results[column]['Levene Statistic']:.3f}, p-value = {homogeneity_results[column]['p-value']:.3f}")

Normality Test Results (Shapiro-Wilk):
PANSS-N(1):
    Group 1: W-statistic=0.986, p-value=0.721
    Group 2: W-statistic=0.951, p-value=0.208
PANSS-P(1):
    Group 1: W-statistic=0.978, p-value=0.389
    Group 2: W-statistic=0.871, p-value=0.003
PANSS-G(1):
    Group 1: W-statistic=0.935, p-value=0.004
    Group 2: W-statistic=0.887, p-value=0.006
PANSS-T(1):
    Group 1: W-statistic=0.947, p-value=0.013
    Group 2: W-statistic=0.825, p-value=0.000
Baseline_Negative:
    Group 1: W-statistic=0.968, p-value=0.127
    Group 2: W-statistic=0.945, p-value=0.144
Baseline_Positive:
    Group 1: W-statistic=0.971, p-value=0.188
    Group 2: W-statistic=0.929, p-value=0.057
Baseline_Affective:
    Group 1: W-statistic=0.961, p-value=0.058
    Group 2: W-statistic=0.972, p-value=0.640
Baseline_Cognitive:
    Group 1: W-statistic=0.957, p-value=0.038
    Group 2: W-statistic=0.884, p-value=0.005

Homogeneity of Variance Test Results (Levene's Test):
PANSS-N(1): Levene Statistic = 0.881, p-valu

In [2]:
import numpy as np

def cohen_d(x, y):
    nx = len(x)
    ny = len(y)
    dof = nx + ny - 2
    return (np.mean(x) - np.mean(y)) / np.sqrt(((nx-1)*np.std(x, ddof=1) ** 2 + (ny-1)*np.std(y, ddof=1) ** 2) / dof)

t_test_results = {}

for column in columns_to_compare:
    t_stat, p_val = stats.ttest_ind(group1[column], group2[column], equal_var=False)  # 使用 Welch's t-test
    
    d_val = cohen_d(group1[column], group2[column])
    
    t_test_results[column] = {
        'Group 1 Mean': group1[column].mean(),
        'Group 1 Std': group1[column].std(ddof=1),
        'Group 2 Mean': group2[column].mean(),
        'Group 2 Std': group2[column].std(ddof=1),
        't-statistic': t_stat,
        'p-value': p_val,
        'd': d_val
    }


print("| Variable | Group 1 Mean ± Std | Group 2 Mean ± Std | t | p | d |")
print("|----------|--------------------|--------------------|---|---|-----------|")
for column, result in t_test_results.items():
    print(
        f"| {column} | "
        f"{result['Group 1 Mean']:.2f} ± {result['Group 1 Std']:.2f} | "
        f"{result['Group 2 Mean']:.2f} ± {result['Group 2 Std']:.2f} | "
        f"{result['t-statistic']:.2f} | "
        f"{result['p-value']:.3f} | "
        f"{result['d']:.2f} |"  
    )

| Variable | Group 1 Mean ± Std | Group 2 Mean ± Std | t | p | d |
|----------|--------------------|--------------------|---|---|-----------|
| PANSS-N(1) | 21.59 ± 6.59 | 21.79 ± 5.99 | -0.14 | 0.889 | -0.03 |
| PANSS-P(1) | 21.64 ± 4.06 | 21.18 ± 5.99 | 0.37 | 0.715 | 0.10 |
| PANSS-G(1) | 39.67 ± 7.39 | 40.39 ± 6.20 | -0.47 | 0.637 | -0.10 |
| PANSS-T(1) | 82.90 ± 13.97 | 83.36 ± 15.33 | -0.13 | 0.894 | -0.03 |
| Baseline_Negative | 8.24 ± 2.70 | 8.71 ± 2.11 | -0.87 | 0.385 | -0.18 |
| Baseline_Positive | 5.99 ± 1.66 | 6.23 ± 2.07 | -0.54 | 0.591 | -0.13 |
| Baseline_Affective | 5.67 ± 0.93 | 5.57 ± 0.78 | 0.53 | 0.598 | 0.11 |
| Baseline_Cognitive | 9.66 ± 1.82 | 9.76 ± 1.61 | -0.24 | 0.815 | -0.05 |


## 2.增加描述统计-推断统计

### 1.一般信息

In [3]:
import pandas as pd

file_path = './ouput/after preprocessing-195.xlsx'
df = pd.read_excel(file_path)

continuous_fields = [
    'PANSS_Negative', 'PANSS_Positive', 'PANSS_Affective',
    'PANSS_Cognitive', 'PANSS-N', 'PANSS-P', 'PANSS-G', 'Age', 'Education_years', 'BMI',
    'SES', 'RPM', 'Frequency of episodes', 'Age of onset', 
    'Illness of duration (months)', 'Illness of duration (years)', 'Dose Equivalent to Olanzapine (mg/d)'
]

categorical_fields = [
    'Gender', 'Ethnic', 'Residence', 'Only_child',
    'Smoking_status', 'Alcohol_consumption', 'Employed',
    'Marital_status', 'First episode', 
    'Family history of psychiatric disorders',"Medication Strategies"
]
followed_up = df[df['follow-up'] == 1]
lost_to_followup = df[df['follow-up'] == 0]
remission = followed_up[followed_up['remissin_outcome'] == 1]
no_remission = followed_up[followed_up['remissin_outcome'] == 2]

def format_categorical_summary(series):
    counts = series.value_counts()
    percentages = series.value_counts(normalize=True).mul(100).round(2)
    return counts.astype(str) + " (" + percentages.astype(str) + "%)"


def calculate_summary(dataframe):
    continuous_summary = dataframe[continuous_fields].agg(['mean', 'std']).transpose()
    continuous_summary_formatted = continuous_summary.apply(lambda row: f"{row['mean']:.2f} ± {row['std']:.2f}", axis=1)
    categorical_summary = pd.concat({field: format_categorical_summary(dataframe[field]) for field in categorical_fields})
    return pd.concat([continuous_summary_formatted, categorical_summary], axis=0)

all_samples_summary = calculate_summary(df)
followed_up_summary = calculate_summary(followed_up)
lost_to_followup_summary = calculate_summary(lost_to_followup)
remission_summary = calculate_summary(remission)
no_remission_summary = calculate_summary(no_remission)

summaries = pd.concat({
    'All Samples': all_samples_summary,
    'Followed Up': followed_up_summary,
    'Lost to Follow-Up': lost_to_followup_summary,
    'Remission': remission_summary,
    'No Remission': no_remission_summary
}, axis=1)

final_table = summaries.reset_index()
final_table.columns = ['Field Name', 'All Samples', 'Followed Up', 'Lost to Follow-Up', 'Remission', 'No Remission']

markdown_table = final_table.to_markdown(index=False)

print(markdown_table)

| Field Name                                     | All Samples    | Followed Up    | Lost to Follow-Up   | Remission      | No Remission   |
|:-----------------------------------------------|:---------------|:---------------|:--------------------|:---------------|:---------------|
| PANSS_Negative                                 | 8.22 ± 2.50    | 8.38 ± 2.53    | 8.09 ± 2.49         | 8.23 ± 2.70    | 8.68 ± 2.13    |
| PANSS_Positive                                 | 6.24 ± 1.75    | 6.07 ± 1.79    | 6.37 ± 1.70         | 6.05 ± 1.68    | 6.13 ± 2.04    |
| PANSS_Affective                                | 5.65 ± 0.86    | 5.63 ± 0.88    | 5.67 ± 0.85         | 5.66 ± 0.92    | 5.58 ± 0.79    |
| PANSS_Cognitive                                | 9.78 ± 1.72    | 9.68 ± 1.74    | 9.86 ± 1.71         | 9.64 ± 1.82    | 9.75 ± 1.61    |
| PANSS-N                                        | 21.42 ± 6.50   | 21.76 ± 6.37   | 21.15 ± 6.62        | 21.53 ± 6.83   | 22.21 ± 5.37   |
| PANSS-P    

#### 正态和方差齐性检验

In [4]:

def check_normality(group):
    stat, p_value = shapiro(group)
    return p_value > 0.05  

def check_variance_equality(group1, group2):
    stat, p_value = levene(group1, group2)
    return p_value > 0.05  
def calculate_p_values(group1, group2, fields):
    p_values = {}
    for field in fields:
        if group1[field].nunique() > 1 and group2[field].nunique() > 1:
            normal1 = check_normality(group1[field])
            normal2 = check_normality(group2[field])
            
            equal_var = check_variance_equality(group1[field], group2[field]) if normal1 and normal2 else False
            
            stat, p_value = ttest_ind(group1[field], group2[field], equal_var=equal_var)
            p_values[field] = p_value
        else:
            p_values[field] = None
    return p_values


#### 连续变量

In [5]:
import pandas as pd
from scipy.stats import ttest_ind

def calculate_p_values(group1, group2, fields):
    p_values = {}
    for field in fields:
        if group1[field].nunique() > 1 and group2[field].nunique() > 1:
            stat, p_value = ttest_ind(group1[field], group2[field], equal_var=False)  # Welch's t-test
            p_values[field] = p_value
        else:
            p_values[field] = None
    return p_values

followup_attrition_p_values = calculate_p_values(followed_up, lost_to_followup, continuous_fields)

remission_no_remission_p_values = calculate_p_values(remission, no_remission, continuous_fields)

final_table['Follow-Up vs. Attrition p-value'] = final_table['Field Name'].map(followup_attrition_p_values)
final_table['Remission vs. No Remission p-value'] = final_table['Field Name'].map(remission_no_remission_p_values)

final_table['Follow-Up vs. Attrition p-value'] = final_table['Follow-Up vs. Attrition p-value'].apply(lambda x: f"{x:.3f}" if pd.notnull(x) else "")
final_table['Remission vs. No Remission p-value'] = final_table['Remission vs. No Remission p-value'].apply(lambda x: f"{x:.3f}" if pd.notnull(x) else "")

markdown_table_with_pvalues = final_table.to_markdown(index=False)

# Print the Markdown table with p-values
print(markdown_table_with_pvalues)

| Field Name                                     | All Samples    | Followed Up    | Lost to Follow-Up   | Remission      | No Remission   | Follow-Up vs. Attrition p-value   | Remission vs. No Remission p-value   |
|:-----------------------------------------------|:---------------|:---------------|:--------------------|:---------------|:---------------|:----------------------------------|:-------------------------------------|
| PANSS_Negative                                 | 8.22 ± 2.50    | 8.38 ± 2.53    | 8.09 ± 2.49         | 8.23 ± 2.70    | 8.68 ± 2.13    | 0.427                             | 0.401                                |
| PANSS_Positive                                 | 6.24 ± 1.75    | 6.07 ± 1.79    | 6.37 ± 1.70         | 6.05 ± 1.68    | 6.13 ± 2.04    | 0.247                             | 0.856                                |
| PANSS_Affective                                | 5.65 ± 0.86    | 5.63 ± 0.88    | 5.67 ± 0.85         | 5.66 ± 0.92    | 5.58 ± 0.79 

#### 分类变量

In [6]:
import pandas as pd
import scipy.stats as stats


file_path = './ouput/after preprocessing-195.xlsx'

df = pd.read_excel(file_path)


followed_up = df[df['follow-up'] == 1]  
lost_to_followup = df[df['follow-up'] == 0]  # 

remission = followed_up[followed_up['remissin_outcome'] == 1]  
no_remission = followed_up[followed_up['remissin_outcome'] == 2] 


from scipy.stats import chi2_contingency, fisher_exact

from scipy.stats import chi2_contingency, fisher_exact

def calculate_p_value(crosstab):

    if crosstab.shape == (2, 2):
        _, p = fisher_exact(crosstab)
    else:
        chi2, p, dof, expected = chi2_contingency(crosstab)
        
        if (expected < 5).sum() > 0:
            chi2, p, dof, expected = chi2_contingency(crosstab, lambda_="log-likelihood")
        else:
            pass
    return p


p_values = {}


binary_fields = ['Gender', 'Ethnic', 'Residence', 'Only_child', 'Employed', 'First episode', 
                 'Family history of psychiatric disorders']

for field in binary_fields:
    crosstab = pd.crosstab(df[field], df['follow-up'])
    p_values[f"{field} (Followed Up vs Lost to Follow-Up)"] = calculate_p_value(crosstab)

for field in binary_fields:
    crosstab = pd.crosstab(followed_up[field], followed_up['remissin_outcome'])
    p_values[f"{field} (Remission vs No Remission)"] = calculate_p_value(crosstab)

ternary_fields = ['Smoking_status', 'Alcohol_consumption']

for field in ternary_fields:
    crosstab = pd.crosstab(df[field], df['follow-up'])
    p_values[f"{field} (Followed Up vs Lost to Follow-Up)"] = calculate_p_value(crosstab)

for field in ternary_fields:
    crosstab = pd.crosstab(followed_up[field], followed_up['remissin_outcome'])
    p_values[f"{field} (Remission vs No Remission)"] = calculate_p_value(crosstab)

quaternary_fields = ['Marital_status']

for field in quaternary_fields:
    crosstab = pd.crosstab(df[field], df['follow-up'])
    p_values[f"{field} (Followed Up vs Lost to Follow-Up)"] = calculate_p_value(crosstab)

for field in quaternary_fields:
    crosstab = pd.crosstab(followed_up[field], followed_up['remissin_outcome'])
    p_values[f"{field} (Remission vs No Remission)"] = calculate_p_value(crosstab)

p_values_df = pd.DataFrame(list(p_values.items()), columns=['Field', 'P-Value'])


print(p_values_df.to_markdown(index=False))


def check_chi2_assumptions(crosstab):
    chi2, _, _, expected = stats.chi2_contingency(crosstab)
    if (expected < 5).sum() > 0:
        print(f"Warning: There are cells with expected frequencies less than 5, chi2 test may not be valid.\n")
        print(f"Expected frequencies:\n{expected}")
    else:
        print("All expected frequencies are greater than 5.")

gender_crosstab = pd.crosstab(df['Gender'], df['follow-up'])
check_chi2_assumptions(gender_crosstab)



| Field                                                                      |   P-Value |
|:---------------------------------------------------------------------------|----------:|
| Gender (Followed Up vs Lost to Follow-Up)                                  | 0.466011  |
| Ethnic (Followed Up vs Lost to Follow-Up)                                  | 0.822311  |
| Residence (Followed Up vs Lost to Follow-Up)                               | 0.0794046 |
| Only_child (Followed Up vs Lost to Follow-Up)                              | 0.538895  |
| Employed (Followed Up vs Lost to Follow-Up)                                | 0.331315  |
| First episode (Followed Up vs Lost to Follow-Up)                           | 0.189997  |
| Family history of psychiatric disorders (Followed Up vs Lost to Follow-Up) | 0.850304  |
| Gender (Remission vs No Remission)                                         | 0.157422  |
| Ethnic (Remission vs No Remission)                                         | 0.464502  |

In [7]:
import pandas as pd
import scipy.stats as stats
def check_chi2_assumptions(crosstab):
    chi2, _, _, expected = stats.chi2_contingency(crosstab)
    if (expected < 5).sum() > 0:
        message = f"Warning: There are cells with expected frequencies less than 5."
        expected_freq = expected
    else:
        message = "All expected frequencies are greater than 5."
        expected_freq = None
    return message, expected_freq

assumption_checks = {}

all_fields = binary_fields + ternary_fields + quaternary_fields

for field in all_fields:
    crosstab = pd.crosstab(df[field], df['follow-up'])
    message, expected_freq = check_chi2_assumptions(crosstab)
    assumption_checks[f"{field} (Followed Up vs Lost to Follow-Up)"] = message
    if expected_freq is not None:
        print(f"Field: {field} (Followed Up vs Lost to Follow-Up)")
        print(f"Expected frequencies:\n{expected_freq}\n")


for field in all_fields:
    crosstab = pd.crosstab(followed_up[field], followed_up['remissin_outcome'])
    message, expected_freq = check_chi2_assumptions(crosstab)
    assumption_checks[f"{field} (Remission vs No Remission)"] = message
    if expected_freq is not None:
        print(f"Field: {field} (Remission vs No Remission)")
        print(f"Expected frequencies:\n{expected_freq}\n")

assumption_checks_df = pd.DataFrame(list(assumption_checks.items()), columns=['Field', 'Chi-squared Assumption Check'])

print(assumption_checks_df.to_markdown(index=False))

Field: First episode (Followed Up vs Lost to Follow-Up)
Expected frequencies:
[[  5.58974359   4.41025641]
 [103.41025641  81.58974359]]

Field: Alcohol_consumption (Followed Up vs Lost to Follow-Up)
Expected frequencies:
[[74.34358974 58.65641026]
 [30.18461538 23.81538462]
 [ 4.47179487  3.52820513]]

Field: Marital_status (Followed Up vs Lost to Follow-Up)
Expected frequencies:
[[64.28205128 50.71794872]
 [26.83076923 21.16923077]
 [17.32820513 13.67179487]
 [ 0.55897436  0.44102564]]

Field: Ethnic (Remission vs No Remission)
Expected frequencies:
[[51.93023256 25.06976744]
 [ 6.06976744  2.93023256]]

Field: First episode (Remission vs No Remission)
Expected frequencies:
[[ 1.34883721  0.65116279]
 [56.65116279 27.34883721]]

Field: Family history of psychiatric disorders (Remission vs No Remission)
Expected frequencies:
[[ 9.44186047  4.55813953]
 [48.55813953 23.44186047]]

Field: Smoking_status (Remission vs No Remission)
Expected frequencies:
[[36.41860465 17.58139535]
 [ 5.39

### 2.药物用量

In [8]:
import pandas as pd


file_path = './rawdata/195.xlsx'

df = pd.read_excel(file_path)


df['Dose of antipsychotic drugs (mg/day)'] = df['Dose of antipsychotic drugs (mg/day)'].astype(str)

def calculate_stats(group):
    monotherapy_df = group.loc[group['Type of antipsychotic medication'].str.count('_') == 0].copy()
    monotherapy_df['Dose of antipsychotic drugs (mg/day)'] = monotherapy_df['Dose of antipsychotic drugs (mg/day)'].astype(float)
    monotherapy_stats = monotherapy_df.groupby('Type of antipsychotic medication').agg(
        使用人数=('Dose of antipsychotic drugs (mg/day)', 'count'),
        最小计量=('Dose of antipsychotic drugs (mg/day)', 'min'),
        平均计量=('Dose of antipsychotic drugs (mg/day)', 'mean'),
        最大计量=('Dose of antipsychotic drugs (mg/day)', 'max')
    ).round(2) 
    monotherapy_stats['剂量范围'] = monotherapy_stats.apply(
        lambda x: f"{x['最小计量']:.2f}, {x['平均计量']:.2f}, {x['最大计量']:.2f}", axis=1
    )
    monotherapy_stats.reset_index(inplace=True)

    combination_therapy_df = group.loc[group['Type of antipsychotic medication'].str.count('_') > 0].copy()
    doses_per_patient = combination_therapy_df['Dose of antipsychotic drugs (mg/day)'].str.split('_').tolist()
    expanded_doses = pd.DataFrame(doses_per_patient, index=combination_therapy_df.index)
    expanded_doses = expanded_doses.stack().reset_index(level=1, drop=True).astype(float)
    expanded_doses.name = 'Dose of antipsychotic drugs (mg/day)'
    combination_therapy_df = combination_therapy_df.drop('Dose of antipsychotic drugs (mg/day)', axis=1)
    combination_therapy_df = combination_therapy_df.join(expanded_doses)
    combination_therapy_stats = combination_therapy_df.groupby('Type of antipsychotic medication').agg(
        使用人数=('ID', 'nunique'),
        最小计量=('Dose of antipsychotic drugs (mg/day)', 'min'),
        平均计量=('Dose of antipsychotic drugs (mg/day)', 'mean'),
        最大计量=('Dose of antipsychotic drugs (mg/day)', 'max')
    ).round(2) 
    combination_therapy_stats['剂量范围'] = combination_therapy_stats.apply(
        lambda x: f"{x['最小计量']:.2f}, {x['平均计量']:.2f}, {x['最大计量']:.2f}", axis=1
    )
    combination_therapy_stats.reset_index(inplace=True)

    monotherapy_stats = monotherapy_stats[['Type of antipsychotic medication', '使用人数', '剂量范围']]
    combination_therapy_stats = combination_therapy_stats[['Type of antipsychotic medication', '使用人数', '剂量范围']]

    return monotherapy_stats, combination_therapy_stats

all_monotherapy_stats, all_combination_therapy_stats = calculate_stats(df)

followed_df = df[df['follow-up'] == 1]
followed_monotherapy_stats, followed_combination_therapy_stats = calculate_stats(followed_df)

lost_df = df[df['follow-up'] == 0]
lost_monotherapy_stats, lost_combination_therapy_stats = calculate_stats(lost_df)

remission_df = df[df['remissin_outcome'] == 1]
remission_monotherapy_stats, remission_combination_therapy_stats = calculate_stats(remission_df)

non_remission_df = df[df['remissin_outcome'] == 2]
non_remission_monotherapy_stats, non_remission_combination_therapy_stats = calculate_stats(non_remission_df)

def print_stats(title, mono_stats, combo_stats):
    print(f"{title} - 单一药物使用情况：")
    print(mono_stats)
    print(f"{title} - 联合药物使用情况：")
    print(combo_stats)
    print("\n")



print_stats("195人组（所有的被试）", all_monotherapy_stats, all_combination_therapy_stats)
print_stats("被追踪组", followed_monotherapy_stats, followed_combination_therapy_stats)
print_stats("流失组", lost_monotherapy_stats, lost_combination_therapy_stats)
print_stats("缓解组", remission_monotherapy_stats, remission_combination_therapy_stats)
print_stats("未缓解组", non_remission_monotherapy_stats, non_remission_combination_therapy_stats)

195人组（所有的被试） - 单一药物使用情况：
   Type of antipsychotic medication  使用人数                     剂量范围
0                      Aripiprazole     9      10.00, 18.33, 30.00
1                       Amisulpride    14  200.00, 514.29, 1000.00
2                    Chlorpromazine     1   250.00, 250.00, 250.00
3                         Clozapine    17    50.00, 230.88, 400.00
4                        Olanzapine    26       2.00, 17.58, 20.00
5                      Paliperidone    11        3.30, 6.85, 12.00
6                       Perospirone     5      18.00, 30.40, 42.00
7                        Quetiapine     7   150.00, 314.29, 600.00
8                       Risperidone    59         2.50, 4.59, 6.00
9                         Sulpiride     5   600.00, 640.00, 800.00
10                      Ziprasidone     5    80.00, 128.00, 160.00
195人组（所有的被试） - 联合药物使用情况：
          Type of antipsychotic medication  使用人数                    剂量范围
0                   Aripiprazole_Clozapine     2    20.00, 69.38, 200.00


In [9]:
def check_totals(group_df, group_mono_stats, group_combo_stats, group_name):
    monotherapy_total = group_mono_stats['使用人数'].sum()

    combination_therapy_total = group_combo_stats['使用人数'].sum()

    total_unique_patients = group_df['ID'].nunique()

    total_patients_check = (monotherapy_total + combination_therapy_total) == total_unique_patients

    print(f"{group_name} - 单一药物使用总人数: {monotherapy_total}")
    print(f"{group_name} - 联合药物使用总人数: {combination_therapy_total}")
    print(f"{group_name} - 字段总人数: {total_unique_patients}")
    print(f"{group_name} - 单一药物和联合药物的使用人数总和是否等于总人数: {total_patients_check}")
    print("---\n")

check_totals(df, all_monotherapy_stats, all_combination_therapy_stats, "195人组（所有的被试）")
check_totals(followed_df, followed_monotherapy_stats, followed_combination_therapy_stats, "被追踪组")
check_totals(lost_df, lost_monotherapy_stats, lost_combination_therapy_stats, "流失组")
check_totals(remission_df, remission_monotherapy_stats, remission_combination_therapy_stats, "缓解组")
check_totals(non_remission_df, non_remission_monotherapy_stats, non_remission_combination_therapy_stats, "未缓解组")

195人组（所有的被试） - 单一药物使用总人数: 159
195人组（所有的被试） - 联合药物使用总人数: 36
195人组（所有的被试） - 字段总人数: 195
195人组（所有的被试） - 单一药物和联合药物的使用人数总和是否等于总人数: True
---

被追踪组 - 单一药物使用总人数: 68
被追踪组 - 联合药物使用总人数: 18
被追踪组 - 字段总人数: 86
被追踪组 - 单一药物和联合药物的使用人数总和是否等于总人数: True
---

流失组 - 单一药物使用总人数: 91
流失组 - 联合药物使用总人数: 18
流失组 - 字段总人数: 109
流失组 - 单一药物和联合药物的使用人数总和是否等于总人数: True
---

缓解组 - 单一药物使用总人数: 50
缓解组 - 联合药物使用总人数: 8
缓解组 - 字段总人数: 58
缓解组 - 单一药物和联合药物的使用人数总和是否等于总人数: True
---

未缓解组 - 单一药物使用总人数: 18
未缓解组 - 联合药物使用总人数: 10
未缓解组 - 字段总人数: 28
未缓解组 - 单一药物和联合药物的使用人数总和是否等于总人数: True
---



## 3.流失样本(n=109)与追踪到的样本(n=86) 基线水平差异检验

### 3.2 连续变量

In [10]:
import pandas as pd
from scipy import stats
from statsmodels.stats.multitest import multipletests


df = pd.read_excel('./ouput/after preprocessing-195.xlsx') 

continuous_variables = [
    'Stroop_incongruent_rt', 'Stroop_congruent_rt', 'Stroop_neutral_rt',
    'Stroop_interference_effect', 'Go_acc', 'Go_rt', 'Nogo_acc',
    'Switch_cost', 'Mixing_cost', 'RM_1750_acc', 'RM_750_acc',
    'DSBT_Span', 'RPM', 'CBT_Span', 'CBT_acc', 'PANSS_Negative',
    'PANSS_Positive', 'PANSS_Affective', 'PANSS_Cognitive', 'PANSS-N',
    'PANSS-P', 'PANSS-G', 'Age', 'Education_years', 'BMI', 'SES',
    'Age of onset', 'Illness of duration (years)', 'IF', 'EFF', 'IUF',
    'ISF', 'US'
]

followed_df = df[df['follow-up'] == 1]
lost_df = df[df['follow-up'] == 0]

results = []

for var in continuous_variables:

    sw_followed = stats.shapiro(followed_df[var].dropna())
    sw_lost = stats.shapiro(lost_df[var].dropna())
    
    if (sw_followed.pvalue > 0.05) and (sw_lost.pvalue > 0.05):
        test_stat, p_value = stats.ttest_ind(followed_df[var].dropna(), lost_df[var].dropna(), equal_var=False)  # Welch's t-test
    else:
        test_stat, p_value = stats.mannwhitneyu(followed_df[var].dropna(), lost_df[var].dropna())
    
    results.append({
        'Variable': var,
        'Test Statistic': test_stat,
        'p-Value': p_value,
        'Mean Difference (followed-lost)': followed_df[var].mean() - lost_df[var].mean(),
        'Test Type': 't-test' if (sw_followed.pvalue > 0.05) and (sw_lost.pvalue > 0.05) else 'Mann-Whitney U'
    })

p_values = [r['p-Value'] for r in results]

rejections, corrected_p_values, _, _ = multipletests(p_values, alpha=0.05, method='fdr_bh')
for i, result in enumerate(results):
    result['Corrected p-Value'] = corrected_p_values[i]
    result['Significant After Correction'] = rejections[i]

corrected_results_df = pd.DataFrame(results)

print(corrected_results_df)

corrected_results_df.to_excel('./ouput/sample_selection_bias_analysis_corrected.xlsx', index=False)

                       Variable  Test Statistic   p-Value  \
0         Stroop_incongruent_rt     5283.000000  0.128038   
1           Stroop_congruent_rt     5504.000000  0.036917   
2             Stroop_neutral_rt     5224.000000  0.170343   
3    Stroop_interference_effect     5217.000000  0.175988   
4                        Go_acc     4148.500000  0.169091   
5                         Go_rt     5005.000000  0.417127   
6                      Nogo_acc     5099.000000  0.292644   
7                   Switch_cost     4410.000000  0.479793   
8                   Mixing_cost        0.232188  0.816687   
9                   RM_1750_acc     4206.000000  0.218887   
10                   RM_750_acc     4281.000000  0.299519   
11                    DSBT_Span     4277.000000  0.284263   
12                          RPM     5337.000000  0.096774   
13                     CBT_Span     4874.500000  0.623042   
14                      CBT_acc     4878.000000  0.621457   
15               PANSS_N

### 3.3 分类变量

### 3.4 特殊处理的变量

## 4.回归掉社会人口后患者与对照组的EF差异检验

In [11]:
df = pd.read_excel('./rawdata/残差结果.xlsx')  

In [12]:
numeric_cols = ['Stroop_incongruent_rt', 'Stroop_congruent_rt', 'Stroop_neutral_rt', 'Stroop_interference effect_rt', 'Go_acc', 'Go_rt', 'Nogo_acc', 'Switch_cost', 'Mixing_cost', 'RM-1,750_acc', 'RM-750_acc', 'DSBT_Span', 'CBT_Span', 'CBT_acc', 'IF', 'EFF', 'IUF', 'ISF', 'US']

In [13]:
import pandas as pd
from scipy.stats import ttest_ind
from numpy import std, mean, sqrt
import numpy as np


def cohens_d(group1, group2):
    """
    Compute Cohen's d.
    
    group1: Series or NumPy array
    group2: Series or NumPy array
    
    returns a float (Cohen's d)
    """
    diff = group1.mean() - group2.mean()

    n1, n2 = len(group1), len(group2)
    var1 = group1.var()
    var2 = group2.var()


    pooled_var = (n1 * var1 + n2 * var2) / (n1 + n2)
    

    d = diff / np.sqrt(pooled_var)
    
    return d


results_df = pd.DataFrame(columns=['Variable', 'SCZ Mean', 'SCZ Std Dev', 'HC Mean', 'HC Std Dev', 't', 'p', 'Cohen\'s d'])


for col in numeric_cols:
    scz = df[df['Group'] == 1][col]
    hc = df[df['Group'] == 2][col]
    
    t_stat, p_val = ttest_ind(scz, hc)
    
    d = cohens_d(scz, hc)
    if p_val < 0.001:
        p_val_str = "p < 0.001"
    else:
        p_val_str = f"p = {p_val:.3f}"
    
    print(f'Variable {col}:\nSCZ group mean±std dev = {scz.mean():.2f} ± {scz.std():.2f}\nHC group mean±std dev = {hc.mean():.2f} ± {hc.std():.2f}\nt = {t_stat:.3f}, {p_val_str}, Cohen\'s d = {d:.3f}\n')
    

    new_row = pd.DataFrame({
        'Variable': [col],
        'SCZ Mean': [scz.mean()],
        'SCZ Std Dev': [scz.std()],
        'HC Mean': [hc.mean()],
        'HC Std Dev': [hc.std()],
        't': [t_stat],
        'p': [p_val_str],
        'Cohen\'s d': [d]
    })

    results_df = pd.concat([results_df, new_row], ignore_index=True)


results_df.to_excel('./ouput/EF回归掉社会人口后是否差异显著.xlsx', index=False)

Variable Stroop_incongruent_rt:
SCZ group mean±std dev = 2.65 ± 15.81
HC group mean±std dev = -2.65 ± 17.25
t = 3.116, p = 0.002, Cohen's d = 0.321

Variable Stroop_congruent_rt:
SCZ group mean±std dev = 2.53 ± 11.00
HC group mean±std dev = -2.53 ± 11.03
t = 4.461, p < 0.001, Cohen's d = 0.459

Variable Stroop_neutral_rt:
SCZ group mean±std dev = 2.18 ± 9.00
HC group mean±std dev = -2.18 ± 8.30
t = 4.891, p < 0.001, Cohen's d = 0.503

Variable Stroop_interference effect_rt:
SCZ group mean±std dev = -76.86 ± 1444.13
HC group mean±std dev = 76.86 ± 1010.08
t = -1.199, p = 0.231, Cohen's d = -0.123

Variable Go_acc:
SCZ group mean±std dev = -0.01 ± 0.02
HC group mean±std dev = 0.01 ± 0.02
t = -7.001, p < 0.001, Cohen's d = -0.720

Variable Go_rt:
SCZ group mean±std dev = 2.17 ± 7.08
HC group mean±std dev = -2.17 ± 6.27
t = 6.318, p < 0.001, Cohen's d = 0.650

Variable Nogo_acc:
SCZ group mean±std dev = -0.00 ± 0.03
HC group mean±std dev = 0.00 ± 0.03
t = -1.313, p = 0.190, Cohen's d = -0.

In [14]:
print("| Variable                  | SCZ的均值 ± 标准差          | HC的均值 ± 标准差          | t       | p         | d       |")
print("|---------------------------|-----------------------------|-----------------------------|---------|-----------|---------|")

for index, row in results_df.iterrows():
    cohens_d = row["Cohen's d"]
    print(f"| {row['Variable']} | {row['SCZ Mean']:.2f} ± {row['SCZ Std Dev']:.2f} | {row['HC Mean']:.2f} ± {row['HC Std Dev']:.2f} | {row['t']:.3f} | {row['p']} | {cohens_d:.3f} |")

| Variable                  | SCZ的均值 ± 标准差          | HC的均值 ± 标准差          | t       | p         | d       |
|---------------------------|-----------------------------|-----------------------------|---------|-----------|---------|
| Stroop_incongruent_rt | 2.65 ± 15.81 | -2.65 ± 17.25 | 3.116 | p = 0.002 | 0.321 |
| Stroop_congruent_rt | 2.53 ± 11.00 | -2.53 ± 11.03 | 4.461 | p < 0.001 | 0.459 |
| Stroop_neutral_rt | 2.18 ± 9.00 | -2.18 ± 8.30 | 4.891 | p < 0.001 | 0.503 |
| Stroop_interference effect_rt | -76.86 ± 1444.13 | 76.86 ± 1010.08 | -1.199 | p = 0.231 | -0.123 |
| Go_acc | -0.01 ± 0.02 | 0.01 ± 0.02 | -7.001 | p < 0.001 | -0.720 |
| Go_rt | 2.17 ± 7.08 | -2.17 ± 6.27 | 6.318 | p < 0.001 | 0.650 |
| Nogo_acc | -0.00 ± 0.03 | 0.00 ± 0.03 | -1.313 | p = 0.190 | -0.135 |
| Switch_cost | 1.52 ± 10.77 | -1.52 ± 8.23 | 3.092 | p = 0.002 | 0.318 |
| Mixing_cost | -11.88 ± 210.33 | 11.88 ± 175.49 | -1.192 | p = 0.234 | -0.123 |
| RM-1,750_acc | -0.03 ± 0.19 | 0.03 ± 0.17 | -3.076 | p 

# 5.全部完成

In [15]:
print("数据分析全部完成！")

数据分析全部完成！
